# Продвинутое машинное обучение: Домашнее задание 2

#### MADE-DS-22, Вадим Сапцов 

## 1. Исходные данные 

Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Тренировочный набор — турниры с dateStart из 2019 года; тестовый — турниры с dateStart из 2020 года.

In [1]:
import json, pickle
import pandas as pd
import numpy as np
from itertools import chain

# Загрузка данных
results = pickle.load(open('results.pkl', 'rb'))
players = pd.DataFrame.from_dict(pickle.load(open('players.pkl', 'rb')), orient='index').set_index('id')
tournaments = pd.DataFrame.from_dict(pickle.load(open('tournaments.pkl', 'rb')), orient='index').set_index('id')

In [2]:
# Разделяем тренировочную и тестовую выборку
tournaments_2019 = tournaments[tournaments['dateStart'].str.startswith('2019-')]
tournaments_2020 = tournaments[tournaments['dateStart'].str.startswith('2020-')]

In [3]:
# Объединяем данные в одном датафрейме
tourn_list, player_list, team_list, mask_list, pos_list  = [], [], [], [], []

for tourn_id in results:
    all_ind = np.concatenate((tournaments_2019.index, tournaments_2020.index))
    if (tourn_id in all_ind):
        for team in results[tourn_id]:
            if 'mask' in team:
                mask = team['mask']
                position = team['position']
                team_id = team['team']['id']
                for member in team['teamMembers']:
                    tourn_list.append(tourn_id)
                    mask_list.append(mask)
                    pos_list.append(position)
                    team_list.append(team_id)
                    player_list.append(member['player']['id'])
                    
df = pd.DataFrame({'tournament': tourn_list,
                   'team': team_list,
                   'position': pos_list,
                   'player': player_list,
                   'mask': mask_list
             })

df = df[~df['mask'].isna()]
df['mask_len'] = df['mask'].str.len()

In [4]:
# Отфильтровываем турниры с несовпадающими масками
same_masks = df.groupby('tournament')['mask_len'].nunique().eq(1)
same_masks = same_masks[same_masks]
df = df[df['tournament'].isin(same_masks.index)]

In [5]:
# Разбиваем по вопросам, чистим от неопределенных ответов
questions = []
for lng in df['mask_len']:
    questions.extend(np.arange(1, lng + 1))
    
all_questions = pd.DataFrame({
    'tournament': np.repeat(df['tournament'], df['mask_len']),
    'team': np.repeat(df['team'], df['mask_len']),
    'position': np.repeat(df['position'], df['mask_len']),
    'player': np.repeat(df['player'], df['mask_len']),
    'quest_num': questions,
    'correct_ans': list(chain.from_iterable(df['mask']))
})
all_questions['tourn_quest'] = all_questions['tournament'].astype(str) + '_' + all_questions['quest_num'].astype(str)

bad_ans = all_questions[all_questions['correct_ans'].isin(['?', 'X'])]
all_questions = all_questions[~all_questions['tourn_quest'].isin(bad_ans['tourn_quest'])]
all_questions.loc[:, 'correct_ans'] = all_questions['correct_ans'].astype(int, copy=False)

## 2. Baseline-модель

Baseline-модель будем строить на основе логистической регрессии. Для построения модели примем, что результаты команды по каждому вопросу относятся к каждому из её игроков.

In [7]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

In [8]:
train = all_questions[all_questions['tournament'].isin(tournaments_2019.index.values)]
test = all_questions[all_questions['tournament'].isin(tournaments_2020.index.values)]
print('Тренировочная выборка: ', train['tournament'].nunique(),
      'Тестовая выборка: ', test['tournament'].nunique())

Тренировочная выборка:  663 Тестовая выборка:  169


In [9]:
# Применяем one-hot преобразование
enc = OneHotEncoder(handle_unknown='ignore')
X_train = enc.fit_transform(train[['player', 'tourn_quest']])
X_test = enc.transform(test[['player', 'tourn_quest']])

y_train = train['correct_ans']
y_test = test['correct_ans']

print(X_train.shape, X_test.shape)

(16126924, 87828) (4148096, 87828)


In [10]:
base_model = LogisticRegression()
base_model.fit(X_train, y_train)

preds = base_model.predict_proba(X_test)[:, 1]

D:\work\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## 3. Оценка качества

В качестве метрики качества на тестовом наборе считаем ранговые корреляции Спирмена и Кендалла между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.
Если вероятность ответа каждого игрока в текущем турнире равна $p_i$, то общая вероятность для команды равна $p = 1 - \prod_i (1 - p_i)$ 

In [14]:
from scipy.stats import spearmanr, kendalltau
pd.options.mode.chained_assignment = None

def corr_calc(pred_test):
    pred_ranks = test[['tournament', 'team']]
    pred_ranks['neg_pred'] = 1 - pred_test
    pred_ranks = pred_ranks.groupby(['tournament', 'team']).prod().reset_index()
    pred_ranks['position_pred'] = pred_ranks.groupby('tournament')['neg_pred'].rank('dense')
    
    true_ranks = test[['tournament', 'team', 'position']].drop_duplicates()
    combined = pd.merge(pred_ranks, true_ranks, on=['tournament', 'team'])

    # расчет и усреднение корреляций внутри каждого турнира
    spearman_corr = []
    kendall_corr = []
    for tour in combined['tournament'].unique():
        curr = combined[combined['tournament'] == tour]

        if len(curr) > 1:
            spearman_corr.append(spearmanr(curr['position'], curr['position_pred'])[0])
            kendall_corr.append(kendalltau(curr['position'], curr['position_pred'])[0])
    return np.mean(spearman_corr), np.mean(kendall_corr)

In [15]:
spearman_corr, kendall_corr = corr_calc(preds)
print('К-т Кендалла:', kendall_corr, '   к-т Спирмена:', spearman_corr)

К-т Кендалла: 0.5928306462319053    к-т Спирмена: 0.748055361429059


## 4. EM-схема

**E-шаг**: для заданных весов игроков и вопросов вычисляем ожидание скрытой переменной вероятности ответа игрока на вопрос при условии параметров модели и ответов.<br>

**М-шаг**: минимизируем критерий log-loss, обучая логистическую регрессию. За начальное приближение возьмем предсказания логистической регрессии.

In [16]:
from scipy.special import logit, expit
from sklearn.linear_model import LinearRegression
from copy import deepcopy
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [17]:
em_train = deepcopy(train)
em_train['player_pred'] = base_model.predict_proba(X_train)[:, 1]
em_model = LinearRegression()
best_spearman = 0

M = 5
EPS = 1e-6
for step in range(M):
    # E-шаг
    em_train['neg_pred'] = 1 - em_train['player_pred']
    em_teams = 1 - em_train.groupby(['tournament', 'team', 'quest_num'])['neg_pred'].prod()
    em_train = em_train.merge(em_teams.rename('team_pred'), left_on=['tournament', 'team', 'quest_num'], right_index=True)
    em_train['hidden_var'] = em_train['player_pred'] / em_train['team_pred']
    em_train['hidden_var'] = np.where(y_train == 0, 0, em_train['hidden_var'])
    em_train['hidden_var'] = np.clip(em_train['hidden_var'], EPS, 1 - EPS)

    # M-шаг 
    em_model.fit(X_train, logit(em_train['hidden_var']))
    em_train['player_pred'] = expit(em_model.predict(X_train))
    em_train = em_train.drop('team_pred',1)

    # Качество
    print('Шаг', step + 1)
    pred_test = expit(em_model.predict(X_test))
    spearman_corr, kendall_corr = corr_calc(pred_test)
    if spearman_corr > best_spearman:
        best_preds = pred_test
        best_coefs = em_model.coef_

    print('К-т Кендалла: ', kendall_corr, 'к-т Спирмена:', spearman_corr)

Шаг 1
К-т Кендалла:  0.6177868949818319 к-т Спирмена: 0.7722388020880716
Шаг 2
К-т Кендалла:  0.6229168687149627 к-т Спирмена: 0.7790069686893311
Шаг 3
К-т Кендалла:  0.624561647497713 к-т Спирмена: 0.780312492300663
Шаг 4
К-т Кендалла:  0.6210403201617783 к-т Спирмена: 0.7769840520990466
Шаг 5
К-т Кендалла:  0.6197325778719255 к-т Спирмена: 0.7758603789107991


Целевые метрики вначале возрастают, но после трех шагов качество снижается. В итоге прирост метрики составил ок. 3 процентных пунктов.

## 5. Рейтинг-лист турниров

In [18]:
# Сортируем по сложности вопросов (из коэффициентов обученной ЕМ-модели)
question_weights = dict(zip(enc.categories_[1], best_coefs[:enc.categories_[1].shape[0]]))

tournaments_ranking = train[['tournament', 'quest_num', 'tourn_quest']].drop_duplicates()
tournaments_ranking['quest_weight'] = tournaments_ranking['tourn_quest'].map(question_weights)
tournaments_ranking = tournaments_ranking.groupby('tournament')['quest_weight'].mean().reset_index()
tournaments_ranking = tournaments_ranking.merge(tournaments[['name']], left_on='tournament', right_index=True)
tournaments_ranking = tournaments_ranking.sort_values(by='quest_weight', ascending=False)

In [19]:
tournaments_ranking.head(15)

,tournament,quest_weight,name
121,5402,3.312742,Триптих. Осень
96,5370,3.254601,Благородный Дон Синхрон
122,5404,3.204628,Кубок МТС
61,5275,3.133883,Январское диминуэндо
120,5401,3.097134,Триптих. Лето
65,5285,3.076730,Гусарская лига. II сезон. IV этап
97,5371,3.070517,Международный Карагандинский Синхрон
67,5303,3.019102,Мемориал Дмитрия Коноваленко
95,5369,3.003693,Благородный Дон
62,5276,2.997114,Уходящая натура


In [20]:
tournaments_ranking.tail(15)

,tournament,quest_weight,name
543,5939,-2.075831,Чемпионат Мира. Этап 1. Группа В
557,5954,-2.139223,Школьная лига. II тур.
645,6131,-2.409808,ДР Земцовского
587,5995,-2.431661,Гран-при Славянки. 6 этап
569,5975,-2.850003,Чемпионат Минска. Лига Б. Тур первый
658,6161,-2.862135,Студенческий чемпионат Тюменской области
620,6071,-2.992938,Кубок Кольской АЭС
581,5989,-3.015722,Чемпионат Узбекистана
540,5936,-3.050166,Школьная лига. I тур.
622,6078,-3.206447,Гефест
